In [1]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [2]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [3]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [4]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [5]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [6]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [7]:
## 
n='Input_A6_001' # 改名稱
##
dire= ['Data/BayesianRidge_Pre_0/','Data/BayesianRidge_Pre_1/','Data/BayesianRidge_Pre_2/','Data/BayesianRidge_Pre_3/','Data/BayesianRidge_Pre_4/','Data/BayesianRidge_Pre_5/','Data/BayesianRidge_Pre_6/','Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_A6_001
Data/BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.016803044155355423
test_rmse: 0.01503496990567916
test_r2: 0.7797425182625118
feature number: 3
train_rmse: 0.015629789219379735
test_rmse: 0.014470729413368688
test_r2: 0.801804072673548
feature number: 4
train_rmse: 0.015609899105454548
test_rmse: 0.014621782901031444
test_r2: 0.7980064080094318
feature number: 5
train_rmse: 0.01560854550156871
test_rmse: 0.014625544182214501
test_r2: 0.7978891207139396
feature number: 6
train_rmse: 0.01557139493742218
test_rmse: 0.01458691384956358
test_r2: 0.7991577179877682
feature number: 7
train_rmse: 0.01548652691912664
test_rmse: 0.01457080678166528
test_r2: 0.7999937231383877
feature number: 8
train_rmse: 0.015474271119849208
test_rmse: 0.014567206913616454
test_r2: 0.8000871168962638
feature number: 9
train_rmse: 0.015403310534757107
test_rmse: 0.01454346502792771
test_r2: 0.8008186277818605
feature number: 10
train_rmse: 0.015390855608910754
test_rmse: 0.0145997652874287

train_rmse: 0.015031730497058715
test_rmse: 0.01460792704777017
test_r2: 0.7985749157501998
feature number: 17
train_rmse: 0.014963898549168094
test_rmse: 0.014759349415940695
test_r2: 0.7929451495497523
feature number: 18
train_rmse: 0.014954062382748379
test_rmse: 0.01482894938695066
test_r2: 0.7910599893146367
feature number: 19
train_rmse: 0.014831553915090648
test_rmse: 0.01474252487529971
test_r2: 0.793621024475306
feature number: 20
train_rmse: 0.014814966124270709
test_rmse: 0.014813819599039749
test_r2: 0.7918656213951061
Test_rmse_min = 1.fea_num: 3  2.rmse: 0.014470729413368688
Test_r2_max = 1. fea_num: 3  2.r2: 0.801804072673548
Data/BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.016803044155355423
test_rmse: 0.01503496990567916
test_r2: 0.7797425182625118
feature number: 3
train_rmse: 0.015629789219379735
test_rmse: 0.014470729413368688
test_r2: 0.801804072673548
feature number: 4
train_rmse: 0.015609899105454548
test_rmse: 0.014621782901031444
test_r2: 0.79800640800

train_rmse: 0.015403310534757107
test_rmse: 0.01454346502792771
test_r2: 0.8008186277818605
feature number: 10
train_rmse: 0.015390044641981343
test_rmse: 0.0145999905786342
test_r2: 0.7991626716107041
feature number: 11
train_rmse: 0.015379122169980126
test_rmse: 0.014613499534976714
test_r2: 0.7987988969678634
feature number: 12
train_rmse: 0.01513173475881709
test_rmse: 0.014535494450003317
test_r2: 0.8006844475705254
feature number: 13
train_rmse: 0.015128993324391687
test_rmse: 0.01455668674493008
test_r2: 0.800146211671794
feature number: 14
train_rmse: 0.015056891944544149
test_rmse: 0.014478160591141893
test_r2: 0.8022893305644766
feature number: 15
train_rmse: 0.015018694878236094
test_rmse: 0.014465586905372585
test_r2: 0.8026040215516044
feature number: 16
train_rmse: 0.01499393526260619
test_rmse: 0.014584301387514136
test_r2: 0.7991750801492543
feature number: 17
train_rmse: 0.014928281762725797
test_rmse: 0.014738958257161433
test_r2: 0.7933264790894512
feature number: 18

# 可以修改第一層的神經元數量

In [11]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(22, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [9]:
fea_num=[2,3,4,5] # 要測試的feature數量 神經元22

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('Data/BayesianRidge_Pre_1/Input_A6_001.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 2
test_rmse_ave: 0.012794028766765822
[0.0032229608518068243, 0.016816267279514213, 0.009073847996010017, 0.02511852847751866, 0.015722248904028825, 0.004484451760220493, 0.01395326485285091, 0.01279618937760257, 0.019072329363661305, 0.007680198804444395]


test_r2_ave: 0.7987179776427471
[0.9840921878996847, 0.7731603715203432, 0.9024588101042031, 0.16014579317681732, 0.8886955052847194, 0.9766933521310907, 0.8659907232488668, 0.7937699353094468, 0.6901653803989782, 0.9520077173533198]


feature number: 3
test_rmse_ave: 0.009333905416815131
[0.00653654465708081, 0.014030671505256269, 0.005339766043880833, 0.005823319273165691, 0.009091360457372554, 0.013353154522608419, 0.006506618179771012, 0.008421557915000294, 0.011330671737852143, 0.01290538987616329]


test_r2_ave: 0.9090543114440013
[0.934566996291831, 0.842087453165337, 0.9662207884537184, 0.9548605026858928, 0.9627830206180058, 0.7933531208720855, 0.970859712501524, 0.9106744827639249, 0.8906462245367699, 0.86